In [1]:
import os
import plotly.express as px
import pandas as pd

In [7]:
import os
import pandas as pd
import plotly.express as px

def count_images_in_folders(root_folder):
    records = []
    
    for subset in ['train', 'test', 'val']:
        subset_path = os.path.join(root_folder, subset)
        
        for category in os.listdir(subset_path):
            category_path = os.path.join(subset_path, category)
            
            if os.path.isdir(category_path):
                num_images = len([f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))])
                
                records.append({'Category': category, 'Count': num_images, 'Dataset': subset})

    return pd.DataFrame(records)

root_folder = "car_make_images"
df = count_images_in_folders(root_folder)

# Sort and plot
df = df.sort_values(['Dataset', 'Count'], ascending=[True, True])
fig = px.bar(df, x='Count', y='Category', color='Dataset', orientation='h', title='Distribution of Car Makes', 
             height=1000, width=800)
fig.show()
